### Proteomic data

In [3]:
import pandas as pd
df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-WHOLE_ORGANISM-integrated.txt', sep='\t', comment="#", header=None, names=["internal_id", "string_external_id", "abundance"], usecols = [0, 1, 2])
df

,internal_id,string_external_id,abundance
0,1849955,9606.ENSP00000317379,54.100
1,1853483,9606.ENSP00000349708,0.140
2,1853484,9606.ENSP00000349709,0.335
3,1853653,9606.ENSP00000350961,2.680
4,1848000,9606.ENSP00000298861,0.507
...,...,...,...
19944,1861401,9606.ENSP00000419160,0.496
19945,1846669,9606.ENSP00000280358,0.396
19946,1852835,9606.ENSP00000344566,0.909
19947,1843856,9606.ENSP00000247170,8.950


# Should I do bootstrapping here? to get std and cv??

In [5]:
import geckopy
model = geckopy.io.read_sbml_ec_model('/home/biodata/aman/data/ecHumanGEM.xml', hardcoded_rev_reactions=False)  # how is this made actually? principle? # parameter for faster loading
print('Human Model is imported ... ') # 8+ mins # why ec model taking more time than the normal one?ODD # 68 min is tooflong

Human Model is imported ... 


In [6]:
import requests, time, pandas as pd

df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-Adult_Lung_Kim_2014_SEQUEST.txt', sep='\t', comment="#", header=None, names=["internal_id", "string_external_id", "abundance"], usecols = [0, 1, 2])
# df = df[:19000] # testing

df["ENSP"] = df["string_external_id"].str.replace("9606.", "", regex=False) # this strips down to just the enseml tag

ensps = df["ENSP"].drop_duplicates().tolist()

all_results = []

job = requests.post(
    "https://rest.uniprot.org/idmapping/run",
    data={
        "from": "Ensembl_Protein",
        "to": "UniProtKB",
        "ids": ensps
    }
).json()

job_id = job["jobId"]

time.sleep(3)

# Get results
result = requests.get(
    f"https://rest.uniprot.org/idmapping/stream/{job_id}"
).json()

all_results.extend(result.get("results", []))

map_df = pd.DataFrame(all_results)
map_df = map_df.rename(columns={"from": "ENSP", "to": "UniProt"})

merged = df.merge(map_df, on="ENSP", how="left")

merged["protein_gecko_id"] = merged["UniProt"].apply(lambda x: f"prot_{x}[c]" if pd.notna(x) else None)

print(merged.head(20))
print("Total rows mapped:", merged["UniProt"].notna().sum())

    internal_id    string_external_id  abundance             ENSP UniProt  \
0       1853414  9606.ENSP00000349259   1467.000  ENSP00000349259  Q01082   
1       1848000  9606.ENSP00000298861      1.940  ENSP00000298861     NaN   
2       1853413  9606.ENSP00000349252      8.560  ENSP00000349252  P20701   
3       1849954  9606.ENSP00000317376     23.800  ENSP00000317376  P82912   
4       1849659  9606.ENSP00000313869      6.370  ENSP00000313869  Q9BQQ3   
5       1858827  9606.ENSP00000383333      0.017  ENSP00000383333  Q14149   
6       1850404  9606.ENSP00000321971     16.800  ENSP00000321971  Q8IXI1   
7       1862019  9606.ENSP00000436005     27.400  ENSP00000436005  Q9Y296   
8       1864209  9606.ENSP00000469669     22.100  ENSP00000469669  M0R1B8   
9       1844723  9606.ENSP00000259791    196.000  ENSP00000259791     NaN   
10      1842424  9606.ENSP00000205061     32.300  ENSP00000205061  Q92896   
11      1855795  9606.ENSP00000362063     18.300  ENSP00000362063  P33240   

In [7]:
prot = merged[["UniProt", "abundance"]].copy()
prot = prot.dropna(subset=["UniProt"])
prot = prot.rename(columns={"abundance": "copies_per_cell"})

prot["protein_gecko_id"] = prot["UniProt"].apply(lambda x: f"prot_{x}[c]")

prot["stdev"] = 0

print(len(prot))

6186


In [8]:
print(prot)

     UniProt  copies_per_cell protein_gecko_id  stdev
0     Q01082         1467.000   prot_Q01082[c]      0
2     P20701            8.560   prot_P20701[c]      0
3     P82912           23.800   prot_P82912[c]      0
4     Q9BQQ3            6.370   prot_Q9BQQ3[c]      0
5     Q14149            0.017   prot_Q14149[c]      0
...      ...              ...              ...    ...
6661  P55196           26.500   prot_P55196[c]      0
6662  Q6P988            3.760   prot_Q6P988[c]      0
6663  Q12837            0.877   prot_Q12837[c]      0
6664  P16989           67.100   prot_P16989[c]      0
6665  O43822            0.243   prot_O43822[c]      0

[6186 rows x 4 columns]


In [9]:
# prot # 39mins!!!
ec_model_exp = geckopy.experimental.from_copy_number(
    model,
    index=prot["protein_gecko_id"],
    cell_copies = prot["copies_per_cell"],
    stdev = prot["stdev"],
    vol=2.3e-12, dens=1.05, water=0.7 # are these close to bio.reality?
) # pipe output to additional log.f?

protein prot_Q01082[c] from measurements was not found in the model
protein prot_P20701[c] from measurements was not found in the model
protein prot_P82912[c] from measurements was not found in the model
protein prot_Q9BQQ3[c] from measurements was not found in the model
protein prot_Q14149[c] from measurements was not found in the model
protein prot_Q8IXI1[c] from measurements was not found in the model
protein prot_Q9Y296[c] from measurements was not found in the model
protein prot_M0R1B8[c] from measurements was not found in the model
protein prot_Q92896[c] from measurements was not found in the model
protein prot_P33240[c] from measurements was not found in the model
protein prot_O14776[c] from measurements was not found in the model
protein prot_Q8WVJ2[c] from measurements was not found in the model
protein prot_Q8N138[c] from measurements was not found in the model
protein prot_Q96CG8[c] from measurements was not found in the model
protein prot_A0A5K1VW95[c] from measurements was

In [10]:
print(f"Number of proteins.r listed in the model: {len(model.proteins)} ")

Number of proteins.r listed in the model: 3195 


In [14]:
print("The extracted model is: ", ec_model_exp.summary())

The extracted model is:  Objective
1.0 biomass_human = 7.4947084470513914e-06

Uptake
------
     Metabolite              Reaction      Flux  C-Number C-Flux
     m01387s[s]      EX_CE2915[e]_REV 1.811E-06         0  0.00%
     m02000s[s]     EX_HC02193[e]_REV 2.146E-05         0  0.00%
 argphearg_s[s]   EX_argphearg[e]_REV 1.029E-06         0  0.00%
     m01371s[s]         EX_atp[e]_REV 0.0003369         0  0.00%
 cysglutrp_s[s]   EX_cysglutrp[e]_REV 7.345E-07         0  0.00%
 glumethis_s[s]       EX_glumethis[e] 9.034E-07         0  0.00%
     m01972s[s]  EX_gluside_hs[e]_REV  1.06E-07         0  0.00%
     m00788s[s]      EX_hLkynr[e]_REV 2.659E-07         0  0.00%
    hmcarn_s[s]      EX_hmcarn[e]_REV 1.852E-05         0  0.00%
 lyslyslys_s[s]   EX_lyslyslys[e]_REV  9.65E-07         0  0.00%
     m02411s[s]      EX_pmtcrn[e]_REV 1.166E-05         0  0.00%
 propropro_s[s]   EX_propropro[e]_REV   6.9E-07         0  0.00%
       pvs_s[s]         EX_pvs[e]_REV 5.054E-07         0  0.0

In [15]:
# optimize
print(ec_model_exp.slim_optimize())

7.4947084470513914e-06


In [16]:
# pkl doesn't work when loading tho
import pickle

with open("ecHumanGEM_dec.pkl", "wb") as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
# 
from geckopy.io import write_sbml_ec_model

write_sbml_ec_model(model, "ecHumanGEM_expanded.xml")

prot_Q8IWW8[c]
prot_P28332[c]
prot_Q8N4Q0[c]
prot_P07327[c]
prot_P40394[c]
prot_P00325[c]
prot_P11766[c]
prot_P08319[c]
prot_P00326[c]
prot_P14550[c]
prot_Q9NR19[c]
prot_Q9H6R3[c]
prot_Q9NUB1[c]
prot_O00330[c]
prot_P08559[c]
prot_P09622[c]
prot_P10515[c]
prot_P11177[c]
prot_P29803[c]
prot_P07195[c]
prot_P00338[c]
prot_Q8IX04[c]
prot_P07864[c]
prot_Q6ZMR3[c]
prot_Q9BYZ2[c]
prot_P43353[c]
prot_P30838[c]
prot_P48448[c]
prot_P47895[c]
prot_P51648[c]
prot_P05091[c]
prot_P30837[c]
prot_P49189[c]
prot_P49419[c]
prot_Q01813[c]
prot_P17858[c]
prot_P08237[c]
prot_O43822[c]
prot_P09972[c]
prot_P05062[c]
prot_P04075[c]
prot_P14618[c]
prot_Q969K7[c]
prot_P30613[c]
prot_Q9P1Z3[c]
prot_P06733[c]
prot_P13929[c]
prot_P09104[c]
prot_P15259[c]
prot_P18669[c]
prot_P07738[c]
prot_Q8N0Y7[c]
prot_P54108[c]
prot_P00558[c]
prot_Q5JRA6[c]
prot_P07205[c]
prot_P07311[c]
prot_P14621[c]
prot_O14556[c]
prot_P04406[c]
prot_O00757[c]
prot_P09467[c]
prot_P06744[c]
prot_P60174[c]
prot_P35557[c]
prot_Q2TB90[c]
prot_P1936